<a href="https://colab.research.google.com/github/Mr-MaNia7/deep-learning/blob/main/DL_Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# import torch
import torch

# Creating Layers

In [63]:
# creating layers

class DenseLayer:
  # Layer initialization with random weights and zero biases
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases


# Activation Functions

## ReLu

In [64]:
# relu
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(torch.tensor(0),inputs)

## Sigmoid

In [65]:
# sigmoid
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))


## Linear Activation

In [66]:
# linear activation
class LinearActivation:
    def forward(self, x):
        self.output = x

# Loss

In [67]:
class Loss_CategoricalCrossentropy() :
  # Forward pass
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss

# Accuracy

In [68]:
class Accuracy():
  def calculate(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    accuracy = torch.mean((predictions == y_true).float())
    return accuracy

## Dataset preparation

In [69]:
X = torch.tensor([0.1, 0.5])
Y = torch.tensor([0.05, 0.95])
print(X)
print(Y)

tensor([0.1000, 0.5000])
tensor([0.0500, 0.9500])


# Creating a model

### Model construction: Input layer, hidden layer with Sigmoid activation, output layer with linear activation

In [70]:
# Use 2 features in the input layer, 1 hidden layer with 4 neurons
hidden_layer_1 = DenseLayer(2, 4)

In [71]:
# Sigmoid activation in the hidden layer.
activation1 = Activation_Sigmoid()

In [72]:
# An output layer with 2 neurons
output_layer = DenseLayer(4, 2)


In [73]:
# linear activation in the output layer.
activation2 = LinearActivation()

# Forward pass

In [74]:
# Forward Pass function
def forward_pass(X):
  hidden_layer_1.forward(X)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation2.forward(output_layer.output)
  return activation2.output

# Backward Propagation

In [75]:
# # Backward propagation
# Backward propagation

def backward_propagation(fp):
    lr = torch.tensor(0.01)

    back1 = (fp[0][0] - Y[0]) * (1 - fp[0][0]) * fp[0][0]
    back2 = (fp[0][1] - Y[1]) * (1 - fp[0][1]) * fp[0][1]

    output_layer.weights[0][0] -= lr * back1 * activation1.output[0][0]
    output_layer.weights[0][1] -= lr * back1 * activation1.output[0][1]
    output_layer.weights[1][0] -= lr * back2 * activation1.output[0][0]
    output_layer.weights[1][1] -= lr * back2 * activation1.output[0][1]
    output_layer.biases[0][0] -= lr * back1
    output_layer.biases[0][1] -= lr * back2

    for i in range(hidden_layer_1.weights.shape[0]):
        hidden_layer_1.weights[i, :] -= lr * (
            (back1 * output_layer.weights[:, i] + back2 * output_layer.weights[:, i]) *
            X[i] * hidden_layer_1.output[0, i] * (1 - hidden_layer_1.output[0, i])
        )

    for j in range(hidden_layer_1.biases.shape[1]):
        hidden_layer_1.biases[0, j] -= lr * (
            (back1 * output_layer.weights[j, 0] + back2 * output_layer.weights[j, 1]) *
            hidden_layer_1.output[0, j] * (1 - hidden_layer_1.output[0, j])
        )


# Error Calculation(MSE)

In [76]:
# Error Calculation(MSE)
def error_calculation(Y_true, Y_pred):
  return torch.mean(0.5*(Y_true - Y_pred)**2)


# Forward and Backward Propagation

In [77]:
# Pre Training
loss = 0.0001

Y_pred = forward_pass(X)
err = error_calculation(Y, Y_pred)
print("Initial Loss:", err)
print("Initial Prediction:",Y_pred)

Initial Loss: tensor(0.2209)
Initial Prediction: tensor([[0.0106, 0.0108]])


In [78]:
# Training
while err > loss:
  backward_propagation(Y_pred)
  Y_pred = forward_pass(X)
  err = error_calculation(Y, Y_pred)

In [79]:
# Post Training
print("Final Loss:", err)
print("Final Prediction:", Y_pred)
print("Target Value:", Y)

Final Loss: tensor(9.9974e-05)
Final Prediction: tensor([[0.0636, 0.9353]])
Target Value: tensor([0.0500, 0.9500])
